<a href="https://colab.research.google.com/github/drshahizan/python-web/blob/main/scrapy/SIX/scrapy_bukukita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir scraping_bukukita

In [2]:
!cd scraping_bukukita

In [3]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 5.4 MB/s eta 0:00:00
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=9b1deacd88c5d03ead6a955399697106774be2cf87e428250a39a2445c63b2b2
  Stored in directory: /root/.cache/pip/wheels/dc/b9/4a/948b1176e084b9e3f85e4ffc3d08f817b1fdf0d973bbb94f81
Successfully built PyDispatcher

In [4]:
!scrapy startproject bukukita

New Scrapy project 'bukukita', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/bukukita

You can start your first spider with:
    cd bukukita
    scrapy genspider example example.com


In [5]:
!scrapy genspider buku bukukita.com

Created spider 'buku' using template 'basic' 


Go to buku.py and paste the code below into the file.

```
from scrapy import Request
import scrapy
import re


class BukuSpider(scrapy.Spider):
    name = 'buku'
    allowed_domains = ['bukukita.com']
    main_url = 'https://www.bukukita.com/katalogbuku.php?page='
    start_urls = [main_url+str(1)]

    def parse(self, response):
        max_page = response.css('ul.pagination a::text').getall()[-1]
        for i in range(1,int(max_page)+1):
            yield Request(self.main_url+str(i),callback=self.next_parse)

    def next_parse(self, response):
        '''Parse the start urls and look for books url'''

        # find all books urls
        catalog = response.css('div.product-grid a::attr(href)')
        for url in catalog:
            url = response.urljoin(url.get())
            yield Request(url,callback=self.find_details)

    def find_details(self, response):
        '''Look for book details information'''

        # find details
        rows = response.css('div.row')
        book = {"source": response.url}
        for row in rows:
            cols = row.css('div[class*=col]')
            if len(cols) == 2:
                key = cols[0].css('::text').get()
                key = re.sub("\s+"," ",key).strip()
                value = cols[1].css('::text').get()
                value = re.sub("\s+"," ",value).strip()
                if key == "Text Bahasa":
                    value = " ".join(re.findall("\w+",value))
                if key != "":
                    book.update({key:value})
        yield book

```




In [6]:
!scrapy runspider buku.py -o output.csv

2023-01-25 17:22:58 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2023-01-25 17:22:58 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.10 (default, Nov 14 2022, 12:59:47) - [GCC 9.4.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.10.147+-x86_64-with-glibc2.29
2023-01-25 17:22:58 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
/usr/local/lib/python3.8/dist-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IM

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/output.csv')

In [9]:
df

,source,Judul,Penulis,No. ISBN,Penerbit,Tanggal terbit,Jumlah Halaman,Berat,Jenis Cover,Dimensi(L x P),Kategori,Bonus,Text Bahasa,Lokasi Stok,Buku sejenis lainnya
0,https://www.bukukita.com/Agama/Islam/167332-Is...,Ishmatul Anbiya: Misi Tanpa Salah Para Pembawa...,NaN,9786237163404,Alifia Books,Februari - 2021,225,200 gr,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,NaN
1,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.bukukita.com/Buku-Novel/Sejarah-Fi...,Buku The Horde: Bagaimana Sebuah Gerombolan Pe...,NaN,9786232201156,Alvabet,Oktober - 2021,480,500 gr,soft cover,-,Sejarah Fiksi,-,Indonesia,Stok Tersedia.,NaN
3,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.bukukita.com/Inspirasi-dan-Spiritu...,Success 101 : Hal-Hal yang Harus Diketahui Ole...,NaN,9786028482912,MIC,Mei - 2010,-,250 gr,Hard Cover,-,Kepemimpinan,-,Indonesia,Stok Tersedia.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,https://www.bukukita.com/Ekonomi-dan-Manajemen...,Success Guaranteed: Garansi Sukses Di Bisnis MLM,NaN,9786020956763,MIC,November - 2018,-,260 gr,Soft Cover,-,Bisnis-Investasi,-,Indonesia,Stok Tersedia.,NaN
244,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,https://www.bukukita.com/Agama/Islam/155871-Ko...,Kompilasi Hadis Dakwah,NaN,9786027973633,Simbiosa Rekatama Media,Maret - 2018,-,250 gr,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,NaN
246,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Data Cleaning**

In [11]:
#change gr to g
df['Berat'] = df['Berat'].str.replace('gr','g', regex=False)
df

,source,Judul,Penulis,No. ISBN,Penerbit,Tanggal terbit,Jumlah Halaman,Berat,Jenis Cover,Dimensi(L x P),Kategori,Bonus,Text Bahasa,Lokasi Stok,Buku sejenis lainnya
0,https://www.bukukita.com/Agama/Islam/167332-Is...,Ishmatul Anbiya: Misi Tanpa Salah Para Pembawa...,NaN,9786237163404,Alifia Books,Februari - 2021,225,200 g,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,NaN
1,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.bukukita.com/Buku-Novel/Sejarah-Fi...,Buku The Horde: Bagaimana Sebuah Gerombolan Pe...,NaN,9786232201156,Alvabet,Oktober - 2021,480,500 g,soft cover,-,Sejarah Fiksi,-,Indonesia,Stok Tersedia.,NaN
3,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.bukukita.com/Inspirasi-dan-Spiritu...,Success 101 : Hal-Hal yang Harus Diketahui Ole...,NaN,9786028482912,MIC,Mei - 2010,-,250 g,Hard Cover,-,Kepemimpinan,-,Indonesia,Stok Tersedia.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,https://www.bukukita.com/Ekonomi-dan-Manajemen...,Success Guaranteed: Garansi Sukses Di Bisnis MLM,NaN,9786020956763,MIC,November - 2018,-,260 g,Soft Cover,-,Bisnis-Investasi,-,Indonesia,Stok Tersedia.,NaN
244,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,https://www.bukukita.com/Agama/Islam/155871-Ko...,Kompilasi Hadis Dakwah,NaN,9786027973633,Simbiosa Rekatama Media,Maret - 2018,-,250 g,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,NaN
246,https://www.bukukita.com/searchresult.php?id=3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#change NAN value to "-"
df.fillna("-")

,source,Judul,Penulis,No. ISBN,Penerbit,Tanggal terbit,Jumlah Halaman,Berat,Jenis Cover,Dimensi(L x P),Kategori,Bonus,Text Bahasa,Lokasi Stok,Buku sejenis lainnya
0,https://www.bukukita.com/Agama/Islam/167332-Is...,Ishmatul Anbiya: Misi Tanpa Salah Para Pembawa...,-,9786237163404,Alifia Books,Februari - 2021,225,200 g,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,-
1,https://www.bukukita.com/searchresult.php?id=3...,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,https://www.bukukita.com/Buku-Novel/Sejarah-Fi...,Buku The Horde: Bagaimana Sebuah Gerombolan Pe...,-,9786232201156,Alvabet,Oktober - 2021,480,500 g,soft cover,-,Sejarah Fiksi,-,Indonesia,Stok Tersedia.,-
3,https://www.bukukita.com/searchresult.php?id=3...,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,https://www.bukukita.com/Inspirasi-dan-Spiritu...,Success 101 : Hal-Hal yang Harus Diketahui Ole...,-,9786028482912,MIC,Mei - 2010,-,250 g,Hard Cover,-,Kepemimpinan,-,Indonesia,Stok Tersedia.,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,https://www.bukukita.com/Ekonomi-dan-Manajemen...,Success Guaranteed: Garansi Sukses Di Bisnis MLM,-,9786020956763,MIC,November - 2018,-,260 g,Soft Cover,-,Bisnis-Investasi,-,Indonesia,Stok Tersedia.,-
244,https://www.bukukita.com/searchresult.php?id=3...,-,-,-,-,-,-,-,-,-,-,-,-,-,-
245,https://www.bukukita.com/Agama/Islam/155871-Ko...,Kompilasi Hadis Dakwah,-,9786027973633,Simbiosa Rekatama Media,Maret - 2018,-,250 g,Soft Cover,-,Islam,-,Indonesia,Stok Tersedia.,-
246,https://www.bukukita.com/searchresult.php?id=3...,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [26]:
# export the dataframe to csv file
df.to_csv('buku_kita_infos.csv', index=False)